<a href="https://colab.research.google.com/github/engelberger/InfGCN-pytorch/blob/main/inference_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip install -q lz4
%pip install -q e3nn==0.5.1
%pip install -q torch==1.13.1
%pip install -q torchvision==0.14.1

In [3]:
%pip install torch-scatter -f https://data.pyg.org/whl/torch-2.3.0+121.html

%pip install torch-cluster==1.6.3
%pip install torch-geometric==2.2.0

Looking in links: https://data.pyg.org/whl/torch-2.3.0+121.html
  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for torch-scatter
  Running setup.py clean for torch-scatter
Failed to build torch-scatter
ERROR: Could not build wheels for torch-scatter, which is required to install pyproject.toml-based projects
  Using cached torch_cluster-1.6.3.tar.gz (54 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for torch-cluster
  Runn

In [4]:
!git clone https://github.com/engelberger/InfGCN-pytorch.git
%cd InfGCN-pytorch
%pip install -q -r requirements.txt

fatal: destination path 'InfGCN-pytorch' already exists and is not an empty directory.
/content/InfGCN-pytorch
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Inference with InfGCN

By Chaoran Cheng, Oct 1, 2023


In [5]:
import os

import torch
import lz4.frame
from tqdm import tqdm
import plotly.graph_objects as go
from IPython.display import Image, display

from datasets import DensityDataset
from models import get_model
from utils import load_config

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device('cuda')

ModuleNotFoundError: No module named 'torch_sparse'

In [ ]:
# Set to True to generate static figures
# Set to False to generate interactive figures
# WARNING: Interactive figures significantly increase the notebook size
static_fig = True

In [ ]:
def get_pretrained_model(cfg_path, model_path):
    print(f'Loading config from {cfg_path}')
    cfg = load_config(cfg_path)
    model = get_model(cfg.model).to(device)
    print(f'Loading model from {model_path}')
    ckpt = torch.load(model_path, map_location=device)
    model.load_state_dict(ckpt['model'])
    return model

In [ ]:
def inference_model(model, g, density, grid_coord, infos, grid_batch_size=8196):
    with torch.no_grad():
        model.eval()
        if grid_batch_size is None:
            preds = model(g.x, g.pos, grid_coord, g.batch, infos).squeeze(0)
        else:
            preds = []
            for grid in tqdm(grid_coord.split(grid_batch_size, dim=1)):
                preds.append(model(g.x, g.pos, grid.contiguous(), g.batch, infos).squeeze(0))
            preds = torch.cat(preds, dim=0)
        diff = torch.abs(preds - density)
        loss = diff.pow(2).sum()
        mae = diff.sum() / density.sum()
    return preds, loss, mae

In [ ]:
def draw_volume(grid, density, atom_type, atom_coord, isomin=0.05, isomax=None, surface_count=5, title=None):
    atom_colorscale = ['grey', 'white', 'red', 'blue', 'green']
    fig = go.Figure()
    fig.add_trace(go.Volume(
        x=grid[..., 0], y=grid[..., 1], z=grid[..., 2],
        value=density,
        isomin=isomin,
        isomax=isomax,
        opacity=0.1, # needs to be small to see through all surfaces
        surface_count=surface_count, # needs to be a large number for good volume rendering
        caps=dict(x_show=False, y_show=False, z_show=False),
    ))
    axis_dict = dict(
        showgrid=False,
        showbackground=False,
        zeroline=False,
        visible=False,
    )
    fig.add_trace(go.Scatter3d(
        x=atom_coord[:, 0],
        y=atom_coord[:, 1],
        z=atom_coord[:, 2],
        mode='markers',
        marker=dict(
            size=10,
            color=atom_type,
            cmin=0, cmax=4,
            colorscale=atom_colorscale,
            opacity=0.6
        )
    ))
    if title is not None:
        title = dict(
            text=title,
            x=0.5, y=0.3,
            xanchor='center',
            yanchor='bottom',
        )
    fig.update_layout(
        autosize=False,
        width=800,
        height=800,
        showlegend=False,
        scene=dict(
            xaxis=axis_dict,
            yaxis=axis_dict,
            zaxis=axis_dict
        ),
        title=title,
        title_font_family='Times New Roman',
    )
    return fig

In [ ]:
# Load the dataset
# If you want to run the pretrained model, you can make a dummy data split file like
# {"train": [], "val": [], "test": []}
dataset = DensityDataset('data/QM9', 'test', 'data_split.json', './atom_info/qm9.json', 'CHGCAR', 'lz4')

In [ ]:
file_id = 24492  # indole
# file_id = 114514  # nonane
# file_id = 214  # benzene
# file_id = 2  # ammonia
with lz4.frame.open(f'data/QM9/{file_id:06d}.CHGCAR.lz4') as f:
    g, density, grid_coord, info = dataset.read_chgcar(f)

g.batch = torch.zeros_like(g.x)
g = g.to(device)
density = density.to(device)
grid_coord = grid_coord.to(device)

In [ ]:
# Display the ground truth electron density
fig = draw_volume(
    grid_coord.detach().cpu().numpy(), density.detach().cpu().numpy(),
    g.x.detach().cpu().numpy(), g.pos.detach().cpu().numpy(),
    isomin=0.05, isomax=3.5, surface_count=5
)
if static_fig:
    img_bytes = fig.to_image(format="png", scale=2)
    display(Image(img_bytes))
else:
    fig.show()

In [ ]:
# Load the pretrained model
# There might be a warning from jitting, which arises inside the `e3nn` package.
# You can safely ignore it.
model = get_pretrained_model('configs/qm9.yml', 'logs/train/40000.pt')
print('Complete!')

In [ ]:
# Make prediction
# The very first step might be slow (due to jitting in `e3nn`), but the rest should be fast.
grid_batch_size = 4096
preds, loss, mae = inference_model(
    model, g, density, grid_coord[None], [info], grid_batch_size=grid_batch_size
)
print(f'Loss: {loss.item():.6f}, MAE: {mae.item():.6f}')

In [ ]:
# Display the electron density difference
fig = draw_volume(
    grid_coord.detach().cpu().numpy(), (density - preds).detach().cpu().numpy(),
    g.x.detach().cpu().numpy(), g.pos.detach().cpu().numpy(), isomin=-0.06, isomax=0.06, surface_count=4
)
if static_fig:
    img_bytes = fig.to_image(format="png", scale=2)
    display(Image(img_bytes))
else:
    fig.show()

In [ ]:
# Display the predicted electron density
fig = draw_volume(
    grid_coord.detach().cpu().numpy(), preds.detach().cpu().numpy(),
    g.x.detach().cpu().numpy(), g.pos.detach().cpu().numpy(), isomin=0.05, isomax=3.5, surface_count=5
)
if static_fig:
    img_bytes = fig.to_image(format="png", scale=2)
    display(Image(img_bytes))
else:
    fig.show()